#### Hybrid: Transformer + Bi-LSTM (Autoencoders)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import psutil
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math

# --- 1. Data loading and preparation ---
data = pd.read_csv('/Users/imperator/Documents/STUDIES/UNIVERSITY OF GHANA/RESEARCH WORK/CORCHIL KELLY KWAME/PORTFOLIO OPTIMIZATION/PROJECT CODE/Dimensionality-Reduction-PortfolioOptimization/EDA/ETFs_datasets/Final_csv/5-cleaned_etf_data.csv')
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True, errors='coerce')
data = data.dropna(subset=['Date'])
data = data.sort_values(['Date', 'Ticker']).reset_index(drop=True)

assets = ['ARKK', 'AGG', 'GLD', 'HYG', 'SPY', 'USO', 'VNQ', 'VXUS']
features = ['returns','RSI_14','Volatility_21','MACD','Beta','Alpha','Volume','DMI_minus_14','DMI_plus_14',
           'Calmar_Ratio_21','ATR_14','Open','Close','High','nonfarm_payrolls','real_gdp','cpi','treasury_10y','building_permits']

df_wide = pd.DataFrame()
for feat in features:
    temp = data.pivot(index='Date', columns='Ticker', values=feat)
    temp.columns = [f"{col}_{feat}" for col in temp.columns]
    df_wide = pd.concat([df_wide, temp], axis=1) if not df_wide.empty else temp
df_wide = df_wide.dropna()

returns_cols = [f"{asset}_returns" for asset in assets]
etf_returns = df_wide[returns_cols]

feature_cols = []
for asset in assets:
    for feat in features:
        if feat != 'returns':
            col = f"{asset}_{feat}"
            if col in df_wide.columns:
                feature_cols.append(col)

features_data = df_wide[feature_cols]

# --- StandardScaler ---
scaler_features = StandardScaler()
features_scaled = scaler_features.fit_transform(features_data.values)

# --- Autoencoder definition ---
latent_dim = 7
input_dim = features_scaled.shape[1]

class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )
    def forward(self, x):
        z = self.encoder(x)
        reconstructed = self.decoder(z)
        return reconstructed, z

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(input_dim, latent_dim).to(device)
features_tensor = torch.tensor(features_scaled, dtype=torch.float32).to(device)

class FeaturesDataset(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        return self.data[idx]

batch_size = 32
epochs = 100
train_dataset = FeaturesDataset(features_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.001)
loss_fn = nn.MSELoss()

# Train autoencoder
autoencoder.train()
for epoch in range(epochs):
    epoch_loss = 0
    for batch_data in train_loader:
        optimizer.zero_grad()
        reconstructed, _ = autoencoder(batch_data)
        loss = loss_fn(reconstructed, batch_data)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

autoencoder.eval()
with torch.no_grad():
    _, features_latent = autoencoder(features_tensor)
features_latent_np = features_latent.cpu().numpy()

window_size = 30

def risk_parity_weights(returns_window):
    cov = np.cov(returns_window.T)
    vol = np.sqrt(np.diag(cov))
    inv_vol = 1.0 / (vol + 1e-8)
    weights = inv_vol / inv_vol.sum()
    return weights

def generate_dynamic_weights_with_risk(returns_df, window=window_size):
    weights_list = []
    for i in range(len(returns_df) - 2*window):
        train_w = returns_df.iloc[i:i+window].values
        test_w = returns_df.iloc[i+window:i+2*window]
        mean_returns = test_w.mean()
        mean_returns[mean_returns < 0] = 0
        r_weights = risk_parity_weights(train_w)
        combined = mean_returns * r_weights
        combined[combined < 0] = 0
        if combined.sum() == 0:
            weights = np.ones(len(mean_returns)) / len(mean_returns)
        else:
            weights = combined / combined.sum()
        weights_list.append(weights)
    return np.array(weights_list)

dynamic_weights = generate_dynamic_weights_with_risk(etf_returns, window=window_size)

def standardize_targets(y):
    scaler = StandardScaler()
    y_scaled = scaler.fit_transform(y.reshape(-1,1)).flatten()
    return y_scaled, scaler

def create_sequences(features_data, returns_data, weights_data, window, clip_sharpe=True):
    X, y_sharpe, y_weights = [], [], []
    limit = len(features_data) - 2*window
    for i in range(limit):
        seq_x = features_data[i:i+window]
        returns_w = returns_data.iloc[i+window:i+2*window]
        mean_ret = returns_w.mean().mean()
        std_ret = returns_w.std().mean()
        sharpe = mean_ret / (std_ret + 1e-6)
        if clip_sharpe:
            sharpe = np.clip(sharpe, -5, 5)
        weight_target = weights_data[i] if i < len(weights_data) else np.ones(len(assets))/len(assets)
        X.append(seq_x)
        y_sharpe.append(sharpe)
        y_weights.append(weight_target)
    y_sharpe_arr = np.array(y_sharpe)
    y_sharpe_scaled, scaler = standardize_targets(y_sharpe_arr)
    return np.array(X), y_sharpe_scaled, np.array(y_weights), scaler

X, y_sharpe_scaled, y_weights, sharpe_scaler = create_sequences(features_latent_np, etf_returns, dynamic_weights, window_size)

class PortfolioDataset(Dataset):
    def __init__(self, X, y_sharpe, y_weights):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y_sharpe = torch.tensor(y_sharpe, dtype=torch.float32).unsqueeze(1)
        self.y_weights = torch.tensor(y_weights, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y_sharpe[idx], self.y_weights[idx]

X_train, X_test, y_sharpe_train, y_sharpe_test, y_weights_train, y_weights_test = train_test_split(
    X, y_sharpe_scaled, y_weights, test_size=0.2, random_state=42)

train_dataset = PortfolioDataset(X_train, y_sharpe_train, y_weights_train)
test_dataset = PortfolioDataset(X_test, y_sharpe_test, y_weights_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach()
        return x

class HybridTransformerBiLSTM(nn.Module):
    def __init__(self, pca_components, n_assets, d_model=64, nhead=8, num_transformer_layers=2, lstm_hidden=64):
        super().__init__()
        self.input_proj = nn.Linear(pca_components, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=256, dropout=0.1)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_transformer_layers)
        self.bilstm = nn.LSTM(input_size=d_model, hidden_size=lstm_hidden, batch_first=True, bidirectional=True)
        self.fc_shared = nn.Linear(lstm_hidden * 2, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.sharpe_out = nn.Linear(128, 1)
        self.weights_out = nn.Linear(128, n_assets)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.input_proj(x)
        x = self.pos_encoder(x)
        x = x.permute(1, 0, 2)
        transformer_output = self.transformer_encoder(x)
        x = transformer_output.permute(1, 0, 2)
        lstm_out, _ = self.bilstm(x)
        last_step = lstm_out[:, -1, :]
        feat = self.relu(self.fc_shared(last_step))
        feat = self.dropout(feat)
        sharpe = self.sharpe_out(feat)
        weights = self.softmax(self.weights_out(feat))
        return sharpe, weights

model = HybridTransformerBiLSTM(
    pca_components=latent_dim,
    n_assets=len(assets),
    d_model=64,
    nhead=4,
    num_transformer_layers=2,
    lstm_hidden=64
).to(device)

criterion_sharpe = nn.MSELoss()
criterion_weights = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 50
model.train()
start_time = time.time()
cpu_start = psutil.cpu_percent(interval=None)

for epoch in range(epochs):
    epoch_loss = 0
    for batch_x, batch_sharpe, batch_weights in train_loader:
        batch_x, batch_sharpe, batch_weights = batch_x.to(device), batch_sharpe.to(device), batch_weights.to(device)
        optimizer.zero_grad()
        pred_sharpe, pred_weights = model(batch_x)
        loss_sharpe = criterion_sharpe(pred_sharpe, batch_sharpe)
        loss_weights = criterion_weights(pred_weights, batch_weights)
        loss = loss_sharpe * 0.1 + loss_weights
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader):.6f}")

train_time = time.time() - start_time
cpu_end = psutil.cpu_percent(interval=None)
cpu_usage = cpu_end - cpu_start

model.eval()
all_pred_sharpe = []
all_true_sharpe = []
all_pred_weights = []
all_true_weights = []

with torch.no_grad():
    for batch_x, batch_sharpe, batch_weights in test_loader:
        batch_x = batch_x.to(device)
        pred_sharpe, pred_weights = model(batch_x)
        all_pred_sharpe.append(pred_sharpe.cpu().numpy())
        all_pred_weights.append(pred_weights.cpu().numpy())
        all_true_sharpe.append(batch_sharpe.cpu().numpy())
        all_true_weights.append(batch_weights.cpu().numpy())

pred_sharpe_scaled = np.vstack(all_pred_sharpe).flatten()
true_sharpe_scaled = np.vstack(all_true_sharpe).flatten()
pred_weights = np.vstack(all_pred_weights)
pred_weights /= pred_weights.sum(axis=1, keepdims=True)
true_weights = np.vstack(all_true_weights)

pred_sharpe = sharpe_scaler.inverse_transform(pred_sharpe_scaled.reshape(-1,1)).flatten()
true_sharpe = sharpe_scaler.inverse_transform(true_sharpe_scaled.reshape(-1,1)).flatten()

test_returns = etf_returns.iloc[-len(pred_weights):].values
portfolio_returns = np.sum(test_returns * pred_weights, axis=1)
turnover = np.mean(np.sum(np.abs(np.diff(pred_weights, axis=0)), axis=1))

def sortino_ratio(returns, risk_free=0, periods_per_year=252):
    returns = pd.Series(returns)
    downside = returns[returns < risk_free]
    expected_return = returns.mean() * periods_per_year
    downside_std = downside.std() * np.sqrt(periods_per_year)
    return expected_return / downside_std if downside_std > 0 else np.nan

def calmar_ratio(returns, period_per_year=252):
    returns = pd.Series(returns)
    cumulative = (1 + returns).cumprod()
    annualized_return = cumulative.iloc[-1] ** (period_per_year / len(returns)) - 1
    max_drawdown = ((cumulative.cummax() - cumulative) / cumulative.cummax()).max()
    return annualized_return / max_drawdown if max_drawdown > 0 else np.nan

mse_sharpe = mean_squared_error(true_sharpe, pred_sharpe)
mae_sharpe = mean_absolute_error(true_sharpe, pred_sharpe)
r2_sharpe = r2_score(true_sharpe, pred_sharpe)
sortino = sortino_ratio(portfolio_returns)
calmar = calmar_ratio(portfolio_returns)

def evaluate_portfolio_performance(portfolio_returns):
    portfolio_returns = pd.Series(portfolio_returns).replace([np.inf, -np.inf], np.nan).dropna()
    periods_per_year = 252
    portfolio_values = (1 + portfolio_returns).cumprod()
    peak = portfolio_values.cummax()
    drawdown = (peak - portfolio_values) / peak
    max_drawdown = drawdown.max()
    total_periods = len(portfolio_returns)
    annualized_return = (portfolio_values.iloc[-1]) ** (periods_per_year / total_periods) - 1
    annualized_volatility = portfolio_returns.std() * np.sqrt(periods_per_year)
    mean_return = portfolio_returns.mean() * periods_per_year
    sharpe_ratio = mean_return / annualized_volatility if annualized_volatility > 0 else np.nan
    return {
        'Sharpe Ratio': sharpe_ratio,
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Max Drawdown': max_drawdown
    }

performance_metrics = evaluate_portfolio_performance(portfolio_returns)

print("\n--- Portfolio Performance with Hybrid Transformer + BiLSTM + Autoencoder ---")
print(f"Training time (seconds): {train_time:.2f}")
print(f"CPU usage change (%): {cpu_usage:.2f}\n")

for metric, value in performance_metrics.items():
    if "Return" in metric or "Volatility" in metric or "Drawdown" in metric:
        print(f"- {metric}: {value*100:.2f}%")
    else:
        print(f"- {metric}: {value:.4f}")

print(f"- Sortino Ratio: {sortino:.4f}")
print(f"- Calmar Ratio: {calmar:.4f}")
print(f"- Average Turnover: {turnover:.4f}")
print(f"- Sharpe Prediction MSE: {mse_sharpe:.6f}")
print(f"- Sharpe Prediction MAE: {mae_sharpe:.6f}")
print(f"- Sharpe Prediction R2: {r2_sharpe:.4f}")

# Save predicted portfolio weights
weights_df = pd.DataFrame(pred_weights, index=etf_returns.index[-len(pred_weights):], columns=assets)
weights_df.to_csv('hybrid_transformer_bilstm_autoencoder_portfolio_weights.csv')
print("Saved predicted portfolio weights to 'hybrid_transformer_bilstm_autoencoder_portfolio_weights.csv'.")
